In [4]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils
import pickle
import h5py


# KDD + and + 
X_testP_ = pd.read_pickle("./dataset/kdd_test_2labels.pkl")
X_trainP_ = pd.read_pickle("./dataset/kdd_train_2labels.pkl")
Y_trainP_ = X_trainP_.iloc[:, 122:].as_matrix()
Y_testP_ = X_testP_.iloc[:, 122:].as_matrix()
X_trainP_ = X_trainP_.as_matrix()
X_testP_ = X_testP_.as_matrix()
scaler1 = Normalizer().fit(X_trainP_[:,:122])
X_trainP_ = scaler1.transform(X_trainP_[:,:122])
scaler2 = Normalizer().fit(X_testP_[:,:122])
X_testP_ = scaler2.transform(X_testP_[:,:122])
X_trainP_ = np.reshape(X_trainP_, (X_trainP_.shape[0],X_trainP_.shape[1],1))
X_testP_ = np.reshape(X_testP_, (X_testP_.shape[0],X_testP_.shape[1],1))
print(Y_testP_.shape)
print(X_testP_.shape)
print(Y_trainP_.shape)
print(X_trainP_.shape)

(22544, 2)
(22544, 122, 1)
(125973, 2)
(125973, 122, 1)


In [5]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(122, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 122, 64)           256       
_________________________________________________________________
activation_1 (Activation)    (None, 122, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3904)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               499840    
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
___________________________________________________________

In [6]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_trainP_, Y_trainP_, batch_size=64, validation_data=(X_testP_, Y_testP_) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("PP.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("PP.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_ValaccuaracyPP.png')

Train on 125973 samples, validate on 22544 samples
Epoch 1/25
125973/125973 [==============================] - 31s 245us/step - loss: 0.1449 - acc: 0.9477 - val_loss: 0.8807 - val_acc: 0.7700
Epoch 2/25
125973/125973 [==============================] - 31s 242us/step - loss: 0.0991 - acc: 0.9644 - val_loss: 0.8810 - val_acc: 0.7686
Epoch 3/25
125973/125973 [==============================] - 29s 232us/step - loss: 0.0845 - acc: 0.9684 - val_loss: 1.1192 - val_acc: 0.7686
Epoch 4/25
125973/125973 [==============================] - 29s 231us/step - loss: 0.0747 - acc: 0.9721 - val_loss: 1.0708 - val_acc: 0.7731
Epoch 5/25
125973/125973 [==============================] - 29s 231us/step - loss: 0.0680 - acc: 0.9747 - val_loss: 1.1395 - val_acc: 0.7663
Epoch 6/25
125973/125973 [==============================] - 30s 240us/step - loss: 0.0609 - acc: 0.9767 - val_loss: 1.1194 - val_acc: 0.7826
Epoch 7/25
125973/125973 [==============================] - 31s 243us/step - loss: 0.0569 - acc: 0.9785